In [1]:
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import re
from nltk.stem.porter import PorterStemmer
import heapq
import json
import gc
porter = PorterStemmer()

In [2]:
stop_words = stopwords.words('english')

In [3]:
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()

class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

In [4]:
BLOCK_SIZE = 10000

In [5]:
def bsbi():
    freq_dict = defaultdict(dict)
    total_files = 0
    i = 0
    current_block = 0
    corpus_size=0
    for doc in json.load(open('cran_data.json')):
        title = doc["title"] 
        author = doc["author"] 
        id=doc["id"] 
        bookshelf=doc["bibliography"] 
        text =doc["body"]
        i += 1
        corpus_size+=1
        
        for word in text.split(" "):
            word_count = text.count(word)
            word = remove_symbols(word)
            if word and word not in stop_words:
                word = porter.stem(word)
                if word not in freq_dict:
                    current_block += 1
                if not freq_dict[word].__contains__(id):
                    freq_dict[word].update({id:word_count})
                    current_block += 1  
            if current_block >= BLOCK_SIZE:
                sorted_list = sorted(freq_dict.items(), key=lambda k: k[0]) 
                with open(f'./PL/OP{total_files}.txt', 'w') as  f:
                    for word_id, docs in sorted_list:
                        f.write(word_id)
                        for doc_id,tf in docs.items():
                            f.write(f'->{doc_id,tf}')
                        f.write('\n')    
                current_block = 0
                freq_dict.clear()
                total_files += 1
                print(i, ' rows done')
      
    sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) 
    if len(sorted_list) > 0:
        with open(f'./PL/OP{total_files}.txt', 'w') as  f:
            for word_id, docs in sorted_list:
                f.write(word_id)
                for doc_id,tf in docs.items():
                    f.write(f'->{doc_id,tf}')
                f.write('\n')
        current_block = 0
        freq_dict.clear()
        total_files += 1
    return [corpus_size,total_files]
[corpus_size,file_count] = bsbi()

124  rows done
240  rows done
373  rows done
522  rows done
655  rows done
784  rows done
923  rows done
1061  rows done
1202  rows done
1320  rows done


In [6]:
file_names = [f'./PL/OP{i}.txt' for i in range(0,file_count)]
file_pointers = [open(i) for i in file_names]

In [7]:
def decorated_file(f, key):
    for line in f:
        yield (key(line), line)

files = map(open, file_names)
outfile = open('./PL/merged.txt', 'w')

def key_fn(line):
    return line.split('->', 2)[0]

prev = ''
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
    if prev != line[0]:
        outfile.write(f'\n{line[1].strip()}')
        prev = line[0]
    else:
        outfile.write(f'{line[1][len(line[0]):].strip()}')

for i in file_pointers:
    i.close()
outfile.close()

In [8]:
import ast,math

outfile = open('./PL/tfidf.txt', 'w')

def tf_IDF():
    N = corpus_size
    file1 = open('./PL/merged.txt', 'r')
    Lines = file1.readlines()
    
    count = 0
    for line in Lines:
        l = (line.strip()).split("->")
        word = l[0]
        idf_word =(math.log((1+N)/(len(l[1:])+1)))+1
        outfile.write(f'\n{l[0].strip()}')
        
        for i in range(1,len(l)):
            tuple_l = []
            if len(l) > 1:
                x = list(ast.literal_eval(l[i]))
                x.append(x[1]*idf_word)
                outfile.write(f'->{tuple(x)}')

tf_IDF()